# 챗봇

In [93]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import gensim

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

## 데이터 

https://github.com/songys/Chatbot_data

In [2]:
df = pd.read_csv("data/ChatbotData.csv")

In [3]:
# 중복 질문 확인
df[df.duplicated("Q") == True]

,Q,A,label
196,고양이 키우고 싶어,가족들과 상의해보세요.,0
235,공시 준비 힘들어,잘 될 거예요.,0
1294,돈 벌고 싶어,많이 벌수록 좋아요.,0
1445,로또 번호 알려줘,알면 제가 하죠.,0
1481,마음이 울적해,거리를 걸어보세요.,0
...,...,...,...
11642,착해서 잘해주는 건지 좋아하는 건지,헷갈린다고 말해보세요.,2
11649,첫 눈에 반하는게 가능해?,당연히 가능하죠.,2
11658,첫사랑 생각나,지금의 사랑에 충실하세요.,2
11732,커플여행이 나을까 그냥 우리끼리 갈까?,저는 둘이 가는 게 좋아요.,2


In [4]:
df[df["Q"] == "고양이 키우고 싶어"]

,Q,A,label
195,고양이 키우고 싶어,자신을 먼저 키우세요.,0
196,고양이 키우고 싶어,가족들과 상의해보세요.,0


In [5]:
def preprocessing(sentence):
    import re

    sentence = sentence.lower()  # 소문자화
    sentence = re.sub(r"[^가-힣1-9a-zA-Z?.!,]+", " ", sentence)  # 기타 문자 제거

    return sentence

In [6]:
def build_corpus(df, length, labels):
    from konlpy.tag import Mecab

    tokenizer = Mecab(dicpath=r"C:\mecab\share\mecab-ko-dic")
    df_copy = df.copy()
    for l in labels:
        df_copy[l] = df_copy[l].apply(lambda x: preprocessing(x))
        df_copy[l] = df_copy[l].apply(lambda x: tokenizer.morphs(x))
        df_copy = df_copy[df_copy[l].apply(len) <= length]
    df_copy = df_copy.drop_duplicates(labels[0])

    return df_copy, tokenizer

In [122]:
df_clean, tokenizer = build_corpus(df, 20, df.columns[:2])

In [123]:
print(
    f"데이터 길이 제한으로 사라진 데이터 수 : {df.shape[0]-df_clean.shape[0]}, {(df.shape[0]-df_clean.shape[0])/df.shape[0]*100}%"
)

데이터 길이 제한으로 사라진 데이터 수 : 328, 2.774253573543094%


In [124]:
# 토큰 추가
df_clean["A"] = df_clean["A"].apply(lambda x: ["<start>"] + x + ["<end>"])

In [37]:
def vectorize(df, NUM_WORDS):
    vectorizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS, filters="")
    corpus = []
    for _, sentence in df.iterrows():
        corpus.extend(sentence[df.columns[0]])
        corpus.extend(sentence[df.columns[1]])
    vectorizer.fit_on_texts(corpus)

    temp_df = df.copy()
    temp_df["Q"] = vectorizer.texts_to_sequences(df["Q"])
    temp_df["A"] = vectorizer.texts_to_sequences(df["A"])
    max_len = max(temp_df["Q"].apply(len).max(), temp_df["A"].apply(len).max())

    Q = tf.keras.preprocessing.sequence.pad_sequences(
        temp_df["Q"], maxlen=max_len, padding="post"
    )
    A = tf.keras.preprocessing.sequence.pad_sequences(
        temp_df["A"], maxlen=max_len, padding="post"
    )

    return Q, A, vectorizer

In [64]:
enc_train, dec_train, vectorizer = vectorize(df_clean, None)

In [152]:
max_len = enc_train.shape[1]

In [41]:
# 총 단어 개수
len(vectorizer.word_index)

6767

In [65]:
BATCH_SIZE = 64
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).batch(
    batch_size=BATCH_SIZE
)

## 모델

트랜스포머

In [114]:
import sys
import os

current_dir = os.path.dirname(os.path.abspath("chatbot.ipynb"))
model_dir = os.path.abspath(os.path.join(current_dir, "../../model"))
sys.path.append(model_dir)

import transformer

In [156]:
import importlib

importlib.reload(transformer)

<module 'transformer' from 'c:\\Users\\zzoon\\projects\\aiffel_camp\\model\\transformer.py'>

In [116]:
example_sentence = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야.",
]

In [117]:
if vectorizer.num_words:
    VOCAB_SZIE = vectorizer.num_words
else:
    VOCAB_SZIE = len(vectorizer.word_index)

model = transformer.Transformer(
    n_layers=2,
    d_model=256,
    n_heads=8,
    d_ff=512,
    src_vocab_size=VOCAB_SZIE,
    tgt_vocab_size=VOCAB_SZIE,
    pos_len=enc_train.shape[1],
)

In [151]:
transformer.train(
    model,
    dataset,
    EPOCH=10,
    enc_tokenizer=vectorizer,
    dec_tokenizer=vectorizer,
    example_sentence=example_sentence,
    type="mecab",
    plot_attention=False,
)

Epoch  1: 100%|██████████| 180/180 [00:09<00:00, 19.27it/s, Loss 2.8085]


Input: 지루하다, 놀러가고 싶어.
Predicted translation: 마음 이 있 었 나 봐요 .
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 마음 이 필요 한 것 같 아요 .
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 사랑 에 는 게 좋 겠 네요 .
Input: 집에 있는다는 소리야.
Predicted translation: 마음 이 좀 더 좋 겠 네요 .


Epoch  2: 100%|██████████| 180/180 [00:04<00:00, 44.44it/s, Loss 2.7389]


Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 아 하 는 게 좋 겠 죠 .
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 마음 이 필요 한 것 같 아요 .
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 사랑 은 선 에 대한 사랑 이 에요 .
Input: 집에 있는다는 소리야.
Predicted translation: 서로 에게 좋 아 하 는 게 좋 겠 어요 .


Epoch  3: 100%|██████████| 180/180 [00:03<00:00, 46.04it/s, Loss 2.5817]


...

Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 아 질 거 예요 .
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 오늘 술 만 의 사정 이 있 나 봐요 .
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 부모 님 이나 봐요 .
Input: 집에 있는다는 소리야.
Predicted translation: 페북 을 가지 고 다시 기다릴 수 있 을 거 예요 .


Epoch  9: 100%|██████████| 180/180 [00:04<00:00, 41.80it/s, Loss 1.1910]


Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 아 하 는지 놀 보 세요 .
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 나 의 나 를 위해 좋 은 소식 에 나 봅니다 .
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 만날 수 있 을 거 예요 .
Input: 집에 있는다는 소리야.
Predicted translation: 상황 이 든 대해 다르 죠 .


Epoch 10: 100%|██████████| 180/180 [00:03<00:00, 45.25it/s, Loss 1.0400]


Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 아 하 는 마음 을 때 가 있 기 도 할 거 예요
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 짝사랑 하 나 봐요 .
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 만날 수 없 는 부분 이 었 을 수 도 있 죠 .
Input: 집에 있는다는 소리야.
Predicted translation: 같이 연락 두 는 것 이 라도 하 지 않 는 거


In [178]:
importlib.reload(transformer)

transformer.eval_bleu(
    model,
    df["Q"][:100],
    df["A"][:100],
    vectorizer,
    vectorizer,
    "mecab",
    max_len,
    verbose=False,
)

100%|██████████| 100/100 [00:42<00:00,  2.37it/s]

Num of Sample: 100
Total Score: 0.3341864783791095
